In [9]:
import pandas as pd
from gql import Client, gql                                                                                                                                                                                                                
from gqlactioncable import ActionCableWebsocketsTransport
import requests

In [7]:
dfK = pd.read_csv(r'config.csv', sep=";")
apiSo = dfK.loc[dfK['key'] == 'sorare','api'][0]

In [10]:
url = 'https://api.sorare.com/graphql'

In [11]:
choiceLeagues = [
    "ligue-1-fr",
    "premier-league-gb-eng",
    "serie-a-it",
    "laliga-santander",
    "bayern-munchen-munchen",
    "eredivisie",
    "jupiler-pro-league"
]

In [35]:
def listClubs(league):

    query_clubs_compet = f"""{{competition(slug:"{league}")
                {{
                    clubs
                    {{
                        nodes
                            {{
                                slug
                                name
                            }}
                        }}
                    }}
                }}"""

    r = requests.post(url, json={'query': query_clubs_compet}, headers={'APIKEY':apiSo})

    return r.json()['data']['competition']['clubs']['nodes']

In [36]:
def listPlayers(clubNameGk):
    
    query_clubs_compet = f"""{{club(slug:"{clubNameGk}")
                {{
                    activePlayers
                    {{
                        nodes
                            {{
                                displayName
                                positionTyped
                                appearances
                                displayName
                                averageScore(type:LAST_FIVE_SO5_AVERAGE_SCORE)
                            }}
                        }}
                    }}
                }}"""

    r = requests.post(url, json={'query': query_clubs_compet}, headers={'APIKEY':apiSo})
    return r.json()

In [37]:
lc = listClubs(choiceLeagues[0])

In [47]:
lc[0]['name']

'Angers SCO'

In [48]:
listTeam = []
for items in lc:

    t = listPlayers(items['slug'])
    df = pd.DataFrame(t['data']['club']['activePlayers']['nodes'])
    df['clubName'] = items['name']
    listTeam.append(df)

In [51]:
dfAll = pd.concat(listTeam)

,displayName,positionTyped,appearances,averageScore
0,Gelson Martins,Midfielder,11,0.0
1,Mamadou Coulibaly,Midfielder,0,0.0
2,Wissam Ben Yedder,Forward,40,47.0
3,Thomas Didillon,Goalkeeper,0,0.0
4,Axel Disasi,Defender,40,42.0
5,Myron Boadu,Forward,12,40.0
6,Nazim Babaï,Defender,0,0.0
7,Soungoutou Magassa,Midfielder,2,0.0
8,Edan Diop,Midfielder,6,30.0
9,Krépin Diatta,Forward,40,35.0
